In [4]:
!pip install langchain langchain-community langchain-core langchain-ollama langchain-huggingface sentence_transformers faiss-cpu langchain-openai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=03c8c7abf6a53998770f16c4353aece60bf807cb851b90799d3c6dcbcbd3d262
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: aiofiles
 

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


# ollama_embeddings = OllamaEmbeddings(model="medllama2:latest")

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

2025-08-26 07:27:08.981973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756193229.162570      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756193229.217852      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import os
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# path to your folder with .md files
folder_path = "/kaggle/input/mitra-md-files"

# text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# iterate over all markdown files
for file_name in os.listdir(folder_path):
    if file_name.endswith(".md"):
        file_path = os.path.join(folder_path, file_name)

        # load markdown with mode & strategy
        loader = UnstructuredMarkdownLoader(
            file_path,
            mode="single",     # parse as a single document
            strategy="fast"    # faster parsing
        )
        file_docs = loader.load()

        # split into chunks
        file_docs_splitted = text_splitter.split_documents(file_docs)

        # create FAISS index
        vectordb = FAISS.from_documents(file_docs_splitted, hf_embeddings)

        # save with filename + _hfembeddings
        db_name = os.path.splitext(file_name)[0] + "_hfembeddings"
        vectordb.save_local(db_name)


In [6]:
from huggingface_hub import snapshot_download

# Download model from HF hub
local_dir = snapshot_download(
    repo_id="sentence-transformers/all-MiniLM-L6-v2",
    local_dir="all-MiniLM-L6-v2"
)

print(f"Model downloaded to: {local_dir}")


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

data_config.json: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

onnx/model_O1.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/91.0M [00:00<?, ?B/s]

train_script.py: 0.00B [00:00, ?B/s]

Model downloaded to: /kaggle/working/all-MiniLM-L6-v2


In [7]:
import shutil

shutil.make_archive("/kaggle/working/all-MiniLM-L6-v2", 'zip', "all-MiniLM-L6-v2")


'/kaggle/working/all-MiniLM-L6-v2.zip'